# OpenDiffIt

Compares old files with new files. Tracks progress from old data to new data. Checks new documents for accessibility.

## User step 1: 
Enter location of .csv files

In [7]:
location_of_old_file = "combine/combine-extracolumns.csv"
location_of_new_data_file = "combine/normal-file.csv"
location_for_updated_file = "example/combine-with-new.csv"

## User step 2: 
Choose ```Kernel > Restart Kernel and Run All Cells...``` from the menu

## OpenDiffIt automation:
Sit back a while and let OpenDiffIt do the magic

### OpenDiffIt: Install prerequisites

In [2]:
!pip install -r ./requirements.txt
import logging
import pandas as pd
import numpy as np
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

### OpenDiffIt: Import old data

If you have no old data to compare you still must have a file with column names that inclue 'url' and 'hash'.

In [8]:
df_old = pd.read_csv(location_of_old_file, dtype=str)

df_old.replace(np.nan, '...', inplace=True)
df_old

FileNotFoundError: [Errno 2] No such file or directory: 'combine/combine-extracolumns.csv'

### OpenDiffIt: Import new data

In [9]:
df_new = pd.read_csv(location_of_new_data_file, dtype=str)

df_new.replace(np.nan, '...', inplace=True)
df_new

FileNotFoundError: [Errno 2] No such file or directory: 'combine/normal-file.csv'

### Validate CSVs for required columns.

In [ ]:
if 'url' not in df_new.columns:
    print("STOP: " + location_of_new_data_file + " missing required 'url' column OR missing a column header OR special chars.")
else:
    print("OK: " + location_of_new_data_file + " seems legit.")

In [ ]:
status = "OK: " + location_of_old_file + " seems legit."

if 'url' not in df_old.columns:
    status = "STOP: " + location_of_old_file + " missing required 'url' column OR missing a column header OR has special chars. "
    
if 'hash' not in df_old.columns:
    status = status + "WARNING: " + location_of_old_file + " missing required 'hash' column to compare against."

print(status)

### OpenDiffIt: Create unique 'hash' for each file

In [ ]:
import requests
import hashlib
import logging

def get_remote_sha_sum(url):
    url = url.split('?')[0]
    print('OpenDiffIt: Hashing file ' + url.rsplit('/', 1)[-1].split('?')[0] + ' ...')
    """ Put remote file in memory and create hash """
    MAXSIZE = 26214400 # 25MB
    response = requests.get(url)
    
    if response.status_code == 200:
        try:
            response.raise_for_status()

            if len(response.content) < MAXSIZE:
                sha1 = hashlib.sha1()
                response = response.content
                sha1.update(response)
                return sha1.hexdigest()
            else:
                logging.info('Skipping %s because  %s MB is really big.' % (url, str(MAXSIZE/819200)))
        except requests.exceptions.HTTPError as e:
            print('1')
            print("%(error)s:" % dict(error=e))
            return e
        #     except requests.exceptions.HTTPError as e:
        #         print('1')
        #         print("%(error)s:" % dict(error=e))
        #         return e
        #     except requests.exceptions.ConnectionError as e2:
        #         print('2')
        #         print("%(error)s:" % dict(error=e2))
        #         return e
        #     except requests.exceptions.SSLError as e3:
        #         print('3')
        #         print("%(error)s:" % dict(error=e3))
        #         return e
        #     except requests.exceptions.NewConnectionError as e4:
        #         print('4')
        #         print("%(error)s:" % dict(error=e4))
        #         return e
        #     except requests.exceptions.SocketError as e5:
        #         print('5')
        #         print("%(error)s:" % dict(error=e5))
        #         return e
        #     except requests.exceptions.MaxRetryError as e6:
        #         print('6')
        #         print("%(error)s:" % dict(error=e6))
        #         return e
    
    else:
        return "Status " + str(response.status_code)
        

In [ ]:
# Create hash for each file
    
df_new['hash'] = df_new['url'].apply(get_remote_sha_sum)
# Discard rows with bad urls
df_new = df_new[~df_new['hash'].str.contains("Error", na=True)]
df_new

### OpenDiffIt: Carry over all columns from the old spreadsheet to the new one

In [ ]:
REQ_COLS = ['diff','comply','notes'] # Required cols
EXPIRED_COLS = ['diff', 'count', 'hash'] # Cols with expired data to exclude
existing_cols = list(set().union(df_old.columns, df_new.columns)) # All cols in both spreads
custom_cols = [col for col in existing_cols if col not in EXPIRED_COLS] # Cols that do not exist
all_cols = list(set().union(existing_cols, REQ_COLS))

for col in all_cols:
    if col not in df_new.columns:
        df_new[col] = "..."
                        
df_new.replace(np.nan, '...', inplace=True)
df_new

### OpenDiffIt: Compare old files to new files using 'hash'

In [ ]:
def compare_to_old(row):
    ''' Compare old hash to new hash and migrate relevent data '''    
    is_old = df_old[ df_old['url'] == row['url'] ]
        
    if is_old.empty: 
        row['diff'] = 'NEW'
        row['comply'] = 'UNKNOWN'
        row['notes'] = 'x'
    elif is_old.iloc[0]['comply'] == 'SKIP':
        row['diff'] = 'SKIP'
        row['comply'] = 'SKIP'
        row['notes'] = 'Not relevent.'
    else:
            
        if (is_old.iloc[0]['hash'] == row['hash']):
            for col in custom_cols:
                row[col] = is_old.iloc[0][col] or 'x'
            row['diff'] = 'SAME'
        elif is_old.iloc[0]['hash'] != row['hash']:
            for col in custom_cols:
                row[col] = is_old.iloc[0][col] or 'x'
            row['diff'] = 'UPDATED'
            row['comply'] = 'UNKNOWN'
            row['notes'] = '...'
        else:
            row['diff'] = 'IDK'
            row['comply'] = 'IDK'
            row['notes'] = '...'
    return row

df_new.apply(compare_to_old, axis=1)
df_new.replace(np.nan, '...', inplace=True)

In [ ]:
# Move specific required columns to the end togther
cols_at_end = REQ_COLS
df_new = df_new[[c for c in df_new if c not in cols_at_end]  + [c for c in cols_at_end if c in df_new]]
df_new.replace(np.nan, '...', inplace=True)
df_new

### OpenDiffIt: Check 'NEW' and 'UPDATED' files for compliance

In [ ]:
import os
import hashlib
import requests
import wget
import logging
from pdfminer3.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer3.pdfdevice import TagExtractor
from pdfminer3.pdfpage import PDFPage
from io import BytesIO
from urllib.parse import unquote
    
    
def detect_tags(url):
    """ Detect if PDF has proper tags """
    
    tags = ["<b\'Part", "<b\'Sect", "<b\'Art", "<b\'Content", "<b\'Index", "<b\'BibEntry", "<b\'Lbl", "<b\'Index", "<b\'Note", "<b\'Reference", "<b\'Figure", "<b\'Artifact", "<b\'ArtifactSpan", "<b\'LBody", "<b\'Normal", "<b\'Heading 1", "<b\'Heading 2", "<b\'H1", "<b\'H2", "<b\'Table", "<b\'P", "\'Annots"]

    clean_url = unquote(url)
    rsrcmgr = PDFResourceManager()
    retstr = BytesIO()
    
    if clean_url.endswith('.pdf'):
    

        try:
            device = TagExtractor(rsrcmgr, retstr, codec='utf-8')
        except UnicodeError as ex:
            device = TagExtractor(rsrcmgr, retstr, codec='ascii')

        file_name = clean_url.rsplit('/', 1)[-1].split('?')[0]

        temp_download_file_location = os.path.join('tmp', file_name)

        if os.path.exists(temp_download_file_location):
            print("OpenDiffIt: " + file_name + " Using local cached file.")

        else:
            try:
                temp_download_file_location = wget.download(clean_url, temp_download_file_location)
                print("OpenDiffIt: " + file_name + " Streaming new file from server...")
            except Exception as ex:
                print(ex)

        try:
            # Open the file
            with open(temp_download_file_location, 'rb') as fp:
                print('OpenDiffIt: Checking ' + file_name + ' for tags...')

                fp_size = os.path.getsize(temp_download_file_location)

                MAXSIZE = 2306866 # 22MB

                if fp_size < MAXSIZE:
                    logging.info('File is less than 22 MB. Try to detect.')

                    if (fp_size < MAXSIZE / 2):
                        maxpages = 2
                    elif (fp_size < MAXSIZE / 4):
                        maxpages = 4
                    elif (fp_size < MAXSIZE / 8):
                        maxpages = 8
                    else:
                        maxpages = 1

                    interpreter = PDFPageInterpreter(rsrcmgr, device)
                    password = ''
                    caching = True
                    pagenos=set()
                    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password, caching=caching, check_extractable=True):
                        interpreter.process_page(page)

                    contents = retstr.getvalue().decode()
                    device.close() # check if these need to be here still context manager stuff
                    retstr.close() # check if these need to be here still

                    try:
                        if any(item in contents for item in tags):
                            return label_comply(contents)
                        else:
                            return 'NO', 'Needs Tagged.'
                    except Exception as ex:
                        print(ex)

                else:
                    print('OpenDiffIt: Too big to scan.')
                    return 'UNKNOWN', 'Too big to scan.'

        except Exception as ex:
            print(ex)
            
    else:
        return 'Not PDF', 'Manually check.'

def label_comply(contents):
    """examine the contents of the file"""
    
    try:
        msg = "Is Tagged. "

        if ("<b'H" in contents):
            msg = msg + " And has a Heading Tag."
            logging.info(msg)                    
            status = 'MAYBE'
            notes = msg

        else:
            msg = msg + " But needs a Heading Tag. Other issues possible."
            logging.info(msg)
            status = 'NO'
            notes = msg

        if ("<b'Table" in contents) and ("<b'TH" not in contents):
            msg = msg + " At least one Table is missing TH."
            logging.info(msg)
            status = 'NO'
            notes = msg

        if "_____" in contents:
            msg = msg + " Probably a Form with issues."
            status = 'NO'
            notes = msg

        if "....." in contents:
            msg = msg + " Probably has a table of contents with issues."
            status = 'NO'
            notes = msg

        return status, msg

    except Exception as ex:
        print(ex)

In [ ]:
df_merged = df_new
same = ['SAME', 'SKIP']
other = ['UPDATED', 'NEW', 'IDK']
df_merged_same = df_merged[df_merged.loc[:,'diff'].isin(same)]
df_merged_other = df_merged[df_merged.loc[:,'diff'].isin(other)]

df_merged_other.loc[:,'comply'],df_merged_other.loc[:,'notes'] = zip(*df_merged_other.loc[:,"url"].map(detect_tags))

In [ ]:
df_final = pd.concat([df_merged_same, df_merged_other]).sort_index(ascending=True)

df_final

### OpenDiffIt: Export dataframe as CSV

In [ ]:
df_final.to_csv(location_for_updated_file, index=False)

### OpenDiffIt: Export XLSX file with color coded conditional formatting

In [ ]:
import csv
import xlsxwriter

def csv_to_xlsx(csv_file):
    """ Convert csv to xlsx with formating """
    
    # Calulate column letter for 'comply' column
    num_of_cols = len(df_final.columns)
    comply_col_index = num_of_cols - 1
    comply_col_letter = chr(ord('@')+comply_col_index)
        
    wb = xlsxwriter.Workbook(csv_file[:-4] + '.xlsx')
    ws = wb.add_worksheet("WS1")    # your worksheet title here
    # ws.insert_textbox('B2', 'Edit using Online Excel in Box!', {'width': 256, 'height': 100})
    ws.insert_textbox('G1', 'Only edit using Online Excel in Box!',
                         {'width': 250,
                          'height': 30,
                          'y_offset': 25,
                          'x_offset': 25,
                          'font': {'bold': True,'color': 'red'},
                          'align': {'vertical': 'middle','horizontal': 'center'},
                          'line': {'color': 'red','width': 1.25,'dash_type': 'square_dot'}})

    formatyellow = wb.add_format({'bg_color':'#FFD960'})
    formatpink = wb.add_format({'bg_color':'#ffc0cb'})
    formatgreen = wb.add_format({'bg_color':'#ccff80'})

    # TODO: Do something with goofy character issues other than ignore errors
    with open(csv_file,'r', encoding='utf-8', errors='ignore') as csvfile:
        """ Convert csv to xlsx with formating """
        table = csv.reader(csvfile)
        i = 0
        # write each row from the csv file as text into the excel file
        # this may be adjusted to use 'excel types' explicitly (see xlsxwriter doc)
        for row in table:
            ws.write_row(i, 0, row)
            i += 1
        ws.conditional_format('A1:XFD1048576', {'type':'formula',
                      'criteria':'=INDIRECT("' + comply_col_letter + '"&ROW())="UNKNOWN"',
                      'format':formatyellow})
        ws.conditional_format('A1:XFD1048576', {'type':'formula',
                      'criteria':'=INDIRECT("' + comply_col_letter + '"&ROW())="MAYBE"',
                      'format':formatyellow})
        ws.conditional_format('A1:XFD1048576', {'type':'formula',
                      'criteria':'=INDIRECT("' + comply_col_letter + '"&ROW())="ALMOST"',
                      'format':formatyellow})
        ws.conditional_format('A1:XFD1048576', {'type':'formula',
                      'criteria':'=INDIRECT("' + comply_col_letter + '"&ROW())="NO"',
                      'format':formatpink})
        ws.conditional_format('A1:XFD1048576', {'type':'formula',
                      'criteria':'=INDIRECT("' + comply_col_letter + '"&ROW())="YES"',
                      'format':formatgreen})
        ws.conditional_format('A1:XFD1048576', {'type':'formula',
                      'criteria':'=INDIRECT("' + comply_col_letter + '"&ROW())="SKIP"',
                      'format':formatgreen})

        ws.set_column(0, 0, 75) # url
        ws.set_column(1, 1, 25) # link text
        ws.freeze_panes(1, 0)
    logging.info('Converted csv to pretty xlsx')
    wb.close()

In [ ]:
csv_to_xlsx(location_for_updated_file)

## User step 3: Get your resulting CSV and XLSX file

Result file is in the following relative location:

In [ ]:
location_for_updated_file